In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/ntu_encoder.zip" "/content/"
!ls -l "/content/"
import zipfile
zip_ref = zipfile.ZipFile("/content/ntu_encoder.zip", 'r')
zip_ref.extractall("/")
zip_ref.close()
!rm "/content/ntu_encoder.zip"

total 12485896
drwx------ 6 root root        4096 Feb 28 12:36 drive
-rw------- 1 root root 12785541301 Feb 28 12:38 ntu_encoder.zip
drwxr-xr-x 1 root root        4096 Feb 18 14:33 sample_data


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import vgg16
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from matplotlib import pyplot
import numpy as np
import sys

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig("/content/drive/MyDrive/Colab Notebooks/vgg16/" + filename + '_plot.png')
	pyplot.close()

In [ ]:
from typing import Sequence
# run the test harness for evaluating a model
def evaluationModele():
  # définition du modèle
  vgg_16 = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in vgg_16.layers[:]:
    layer.trainable = False
  for layer in vgg_16.layers:
    print(layer, layer.trainable)
  model = Sequential()
  model.add(vgg_16)
  model.add(Flatten())
  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(120, activation='softmax'))
  model.summary()
  model.compile(loss='categorical_crossentropy',
                optimizer=optimizers.RMSprop(learning_rate=1e-4),
                metrics=['accuracy'])
  ###
  train_batch_size=180
  test_batch_size=18
  train_datagen = ImageDataGenerator()
  test_datagen = ImageDataGenerator()
  # l'iterateur dans le dossier d'entrainement, génère des lots d'image augmentées et normalisées, le batch size est à 16 car on a assez peu d'images
  train_it = train_datagen.flow_from_directory('/content/ntu_encoder/train/',
    class_mode='categorical', batch_size=train_batch_size, target_size=(224, 224))
  print(train_it.class_indices)
  test_it = test_datagen.flow_from_directory('/content/ntu_encoder/test/',
    class_mode='categorical', batch_size=test_batch_size, target_size=(224, 224),shuffle=False)
  # entrainement du modèle
  #checkpoint = ModelCheckpoint("/content/vgg16_1.h5", monitor="val_accuracy", verbose=0,save_best_only=True,save_weights_only=False,mode='auto',save_freq=1)
  #early = EarlyStopping(monitor='val_accuracy',min_delta=0,patience=20,verbose=0,mode='auto')

  hist=model.fit(train_it,steps_per_epoch=train_it.samples/train_it.batch_size,validation_data=test_it,validation_steps=test_it.samples/test_it.batch_size,epochs=20,callbacks=[checkpoint,early], verbose=1)
  # evaluation du modèle
  _, acc = model.evaluate(test_it, steps=len(test_it), verbose=1)
  print('> %.3f' % (acc * 100.0))
  # learning curves
  summarize_diagnostics(hist)
  return model

In [ ]:
model = evaluationModele()
model.save_weights("/content/drive/MyDrive/Colab Notebooks/vgg16/modelNtu120.h5")

Streaming output truncated to the last 5000 lines.
16358/26934 [=================>............] - ETA: 46:22 - loss: 3.0322 - accuracy: 0.3115

In [ ]:
model.save_model()